# 1. Audio files preview 
1) All audio files are loaded from Google Cloud Storage

2) Please forgive me if my Cantonese sounds not native

In [1]:
import IPython
import librosa
import soundfile as sf
import io
from google.cloud import storage
from google.cloud import speech
import os

# get Google Cloud Credential in .json format
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='rich-suprstate-311107-b521ee51d373.json'

BUCKET = 'speech_cantonese_bucket1'

# create a Cloud Storage client
gcs = storage.Client()

# get the bucket from Google Cloud Storage
bucket = gcs.get_bucket(BUCKET)

# specify a filename
file_name = '1.wav'

# read a blob
blob = bucket.blob(file_name)
file_as_string = blob.download_as_string()

# convert the string to bytes and then finally to audio samples as floats 
# and the audio sample rate
data, sample_rate = sf.read(io.BytesIO(file_as_string))

# enable play button in datalab notebook
print ("Display an audio example:")
IPython.display.Audio(data, rate=sample_rate)

Display an audio example:


# 2. Speech to Text

In [2]:
client = speech.SpeechClient()

In [3]:
def transcribe2_gcs(gcs_uri):
    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=44100,
        audio_channel_count=1, # based on the number of channels in the audio file
        enable_separate_recognition_per_channel=False, # Only 1 speaker so no need to separate speech per channel
        language_code='yue-Hant-HK')
    response = client.long_running_recognize(config = config, audio = audio)
    print('Waiting for operation to complete...')
    response = response.result(timeout=36000)

    return response

### Transcribe in batch

In [4]:
gcs_uri = ""
Text_list = []
Confidence_list = []

for i in range(1,51):
    gcs_uri = "gs://" + BUCKET + '/' + str(i) + ".wav"
    res = transcribe2_gcs(gcs_uri)
    print('Completed transcibing Audio No.', i)
    
    # save the text and confidence result into Text_list and Confidence_list
    for result in res.results: 
        Text_list.append('{}'.format(result.alternatives[0].transcript))
        Confidence_list.append(result.alternatives[0].confidence)
#         # print the text result
#         print('Transcript: {}'.format(result.alternatives[0].transcript))
#         print('confidence: {:.3f}'.format(result.alternatives[0].confidence))
#         print()

Waiting for operation to complete...
Completed transcibing Audio No. 1
Waiting for operation to complete...
Completed transcibing Audio No. 2
Waiting for operation to complete...
Completed transcibing Audio No. 3
Waiting for operation to complete...
Completed transcibing Audio No. 4
Waiting for operation to complete...
Completed transcibing Audio No. 5
Waiting for operation to complete...
Completed transcibing Audio No. 6
Waiting for operation to complete...
Completed transcibing Audio No. 7
Waiting for operation to complete...
Completed transcibing Audio No. 8
Waiting for operation to complete...
Completed transcibing Audio No. 9
Waiting for operation to complete...
Completed transcibing Audio No. 10
Waiting for operation to complete...
Completed transcibing Audio No. 11
Waiting for operation to complete...
Completed transcibing Audio No. 12
Waiting for operation to complete...
Completed transcibing Audio No. 13
Waiting for operation to complete...
Completed transcibing Audio No. 14
W

### Save the transcriptions to dataframe

In [5]:
import pandas as pd
Text_df = pd.DataFrame({'Audio_Number' : list(range(1,51)),
                        'Text_GoogleAPI' : Text_list,
                        'Confidence' : Confidence_list,
                        })
Text_df.head()

,Audio_Number,Text_GoogleAPI,Confidence
0,1,係呀但我得唔得閒呀可唔可以遲啲再打畀我吖,0.939040
1,2,我唔需要藉錢啦唔好再打畀我啦,0.939353
2,3,我唔係呀你打錯呀,0.947466
3,4,我唔鍾意,0.950702
4,5,冇興趣拜拜,0.927621


In [6]:
Text_df.Confidence.mean()

0.9338040578365326

# 3. Sentiment Analysis

In [7]:
import argparse

# install package
# pip install --upgrade google-cloud-language

# imports the Google Cloud client library
from google.cloud import language_v1

In [8]:
# define function for printing out sentiment annotations
def print_result(annotations):
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude
    print(
        "Sentiment Score: {}, Magnitude: {}".format(score, magnitude)
    )
    return 0

In [9]:
# function for analyzing sentiment from short-sentence text
def analyze_sentiment(text):
    client = language_v1.LanguageServiceClient()

    document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)
    annotations = client.analyze_sentiment(request={'document': document})

    return annotations

### Save the Sentiment Scores to dataframe

In [11]:
Text_df['Sentiment_Score']= None
Text_df['Sentiment_Magnitude'] = None

for n in range(0,50):
    analyze_sentiment(Text_df['Text_GoogleAPI'][n]).document_sentiment.score
    
    Text_df['Sentiment_Score'][n] = analyze_sentiment(Text_df['Text_GoogleAPI'][n]).document_sentiment.score
    Text_df['Sentiment_Magnitude'][n] = analyze_sentiment(Text_df['Text_GoogleAPI'][n]).document_sentiment.magnitude

/Users/chuanyu/opt/anaconda3/envs/py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Users/chuanyu/opt/anaconda3/envs/py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
Text_df.to_excel("Speech_to_Text_Result_py37.xlsx")
Text_df.head()

,Audio_Number,Text_GoogleAPI,Confidence,Sentiment_Score,Sentiment_Magnitude
0,1,係呀但我得唔得閒呀可唔可以遲啲再打畀我吖,0.939040,-0.3,0.3
1,2,我唔需要藉錢啦唔好再打畀我啦,0.939353,-0.5,0.5
2,3,我唔係呀你打錯呀,0.947466,-0.3,0.3
3,4,我唔鍾意,0.950702,0.2,0.2
4,5,冇興趣拜拜,0.927621,-0.9,0.9


In [13]:
Text_df.describe()

,Audio_Number,Confidence
count,50.00000,50.000000
mean,25.50000,0.933804
std,14.57738,0.031748
min,1.00000,0.774840
25%,13.25000,0.933858
50%,25.50000,0.944911
75%,37.75000,0.950702
max,50.00000,0.950702


# Appendix

### Ap.1 Load file from local drive 

In [74]:
# Load file from local drive (instead of Cloud Storage) and then use Google API
from google.cloud import speech_v1p1beta1 as speech
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='rich-suprstate-311107-b521ee51d373.json'

client = speech.SpeechClient()
encoding=speech.enums.RecognitionConfig.AudioEncoding

input = "stt_success/audio.raw"

config = {
    "language_code": 'en-US',
    "encoding": encoding.LINEAR16,
    "sample_rate_hertz": 22050,
    "speech_contexts": []
}

with open(input, "rb") as f:
    audio = {"content": f.read()}
    
response = client.recognize(config, audio)

for result in response.results: 
    print('Transcript: {}'.format(result.alternatives[0].transcript))
    print('confidence: {:.3f}'.format(result.alternatives[0].confidence))

Transcript: how old is the Brooklyn Bridge
confidence: 0.959


### Ap.2 Simple Example of doing sentiment analysis

In [23]:
# Simple Example - Sentiment Analysis for Cantonese text

# Instantiates a client
client = language_v1.LanguageServiceClient()

# The text to analyze
text = u"冇興趣拜拜"
document = language_v1.Document(content=text,type_=language_v1.Document.Type.PLAIN_TEXT)# language = 'zh-hant-hk'


# Detects the sentiment of the text
annotations = client.analyze_sentiment(request={'document': document})
sentiment = annotations.document_sentiment

print("Text: {}".format(text))
print("Sentiment: {}, {}".format(sentiment.score, sentiment.magnitude))
print(annotations)

Text: 冇興趣拜拜
Sentiment: -0.8999999761581421, 0.8999999761581421
document_sentiment {
  magnitude: 0.8999999761581421
  score: -0.8999999761581421
}
language: "zh"
sentences {
  text {
    content: "\345\206\207\350\210\210\350\266\243\346\213\234\346\213\234"
    begin_offset: -1
  }
  sentiment {
    magnitude: 0.8999999761581421
    score: -0.8999999761581421
  }
}



In [8]:
# Simple Example - Sentiment Analysis for English text

# instantiates a client
client = language_v1.LanguageServiceClient()

# type in the text to analyze
text = u"so far so good"
document = language_v1.Document(content=text, type_=language_v1.Document.Type.PLAIN_TEXT)

# detects the sentiment of the text
annotations = client.analyze_sentiment(request={'document': document})
sentiment = annotations.document_sentiment

print("Text: {}".format(text))
print("Sentiment: {}, {}".format(sentiment.score, sentiment.magnitude))
print(annotations)

Text: so far so good
Sentiment: 0.800000011920929, 0.800000011920929
document_sentiment {
  magnitude: 0.800000011920929
  score: 0.800000011920929
}
language: "en"
sentences {
  text {
    content: "so far so good"
    begin_offset: -1
  }
  sentiment {
    magnitude: 0.800000011920929
    score: 0.800000011920929
  }
}



### Ap.3 Sentiment analysis for multt-sentence texts 

In [ ]:
# [START language_sentiment_tutorial_print_result]
def print_result(annotations):
    score = annotations.document_sentiment.score
    magnitude = annotations.document_sentiment.magnitude

    for index, sentence in enumerate(annotations.sentences):
        sentence_sentiment = sentence.sentiment.score
        print(
            "Sentence {} has a sentiment score of {}".format(index, sentence_sentiment)
        )

    print(
        "Overall Sentiment: score of {} with magnitude of {}".format(score, magnitude)
    )
    return 0
# [END language_sentiment_tutorial_print_result]


# [START language_sentiment_tutorial_analyze_sentiment]
def analyze(movie_review_filename):
    """Run a sentiment analysis request on text within a passed filename."""
    client = language_v1.LanguageServiceClient()

    with open(movie_review_filename, "r") as review_file:
        # Instantiates a plain text document.
        content = review_file.read()

    document = language_v1.Document(content=content, type_=language_v1.Document.Type.PLAIN_TEXT)
    annotations = client.analyze_sentiment(request={'document': document})

    # Print the results
    print_result(annotations)
# [END language_sentiment_tutorial_analyze_sentiment]